In [ ]:
import sys
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import Quorum
from keras.datasets import mnist
from keras.utils import np_utils
from keras.activations import relu
from keras.layers.advanced_activations import PReLU
from keras import backend as K
from keras.callbacks import Callback

def step(X):
    return K.switch(X <= 0, 0, 1)

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

def pushstd(string):
    sys.stdout.write(string)
    sys.stdout.flush()
        
def get_activation_layer(name):
    name = name.lower()
    if name == "quorum":
        return Quorum([step, relu])
    elif name == "relu":
        return Activation('relu')
    elif name == "prelu":
        return PReLU()
    else:
        raise RuntimeError("Activation not known: {}".format(name))

def get_new_model(activation_name):
    model = Sequential()
    model.add(Convolution2D(4, 3, 3, border_mode='valid', init='he_normal', input_shape=(1, 28, 28)))
    model.add(get_activation_layer(activation_name))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, init='he_normal'))
    model.add(get_activation_layer(activation_name))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd')
    return model

def test_model(activation_name, nb_epoch):
    model = get_new_model(activation_name)
    hist = model.fit(X_train, Y_train, batch_size=128, nb_epoch=nb_epoch, 
              verbose=1, validation_data=(X_test, Y_test), show_accuracy=True)
    return hist.history

import os
import cPickle
from collections import defaultdict

nb_epoch=2
activations_to_try = ["quorum", "relu", "prelu"]

if os.path.exists("./results.pkl"):
    results = cPickle.load(open("results.pkl"))
    s = len(results[activations_to_try[0]+"-loss"])
else:
    results = defaultdict(list)
    s = 0

while True:
    pushstd("-- Round {} --\n".format(s))
    for activation in activations_to_try:
        pushstd("Processing {}...\n".format(activation))
        hist = test_model(activation, nb_epoch)
        print(hist)
        print("Adding {}".format(hist['loss'][-1]))
        results[activation+'-loss'].append(hist['loss'][-1])
        results[activation+'-val_loss'].append(hist['val_loss'][-1])
        results[activation+'-acc'].append(hist['acc'][-1])
        results[activation+'-val_acc'].append(hist['val_acc'][-1])
    cPickle.dump(results, open("results.pkl", "wb"))
    s=s+1

-- Round 0 --
Processing quorum...
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 10s - loss: 1.1094 - acc: 0.6742 - val_loss: 0.3599 - val_acc: 0.8926
Epoch 2/2
60000/60000 [==============================] - 10s - loss: 0.5875 - acc: 0.8158 - val_loss: 0.2910 - val_acc: 0.9169
{'acc': [0.67420000000000002, 0.8157833333333333], 'loss': [1.1094361271540323, 0.587472970422109], 'batch': [233, 233], 'val_acc': [0.89259999999999995, 0.91690000000000005], 'val_loss': [0.35987080469131472, 0.29097142341136933], 'size': [127, 127]}
Adding 0.587472970422
Processing relu...
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
43648/60000 [====================>.........] - ETA: 2s - loss: 1.6421 - acc: 0.4463

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/Library/Python/2.7/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/Library/Python/2.7/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/Users/claymcleod/Library/Python/2.7/lib/python/site-packages/ipykernel/kernelbase.py", line 260, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/Users/claymcleod/Library/Python/2.7/lib/python/site-packages/ipykernel/kernelbase.py", line 212, in dispatch_shell
    handler(stream, idents, msg)
  File "/Users/claymcleod/Library/Python/2.7/lib/python/site-packages/ipykernel/kernelbase.py", line 379, in execute_request
    time.sleep(self._execute_sleep)
KeyboardInterrupt
ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/Library/Python/2.7/site-packages/zmq/eventl

In [5]:
import cPickle
results = cPickle.load(open("results.pkl"))
print(results)

defaultdict(<type 'list'>, {'prelu-loss': [1.7119098837534588, 0.77254906800587975], 'quorum-loss': [1.1500703577995299, 0.60095114167531327], 'relu-val_loss': [0.57483391823768615, 0.36985750083923341], 'quorum-acc': [0.64506666666666668, 0.81208333333333338], 'prelu-val_acc': [0.8367, 0.90369999999999995], 'relu-acc': [0.47910000000000003, 0.76028333333333331], 'quorum-val_loss': [0.40019717378616332, 0.32730438485145569], 'prelu-val_loss': [0.6841413105964661, 0.37654474201202393], 'relu-val_acc': [0.85719999999999996, 0.90049999999999997], 'relu-loss': [1.529423282368978, 0.73939602333704635], 'quorum-val_acc': [0.87890000000000001, 0.90380000000000005], 'prelu-acc': [0.40783333333333333, 0.74783333333333335]})
